In [55]:
import pandas as pd

URL = 'https://raw.githubusercontent.com/ajduberstein/oakland_point_cloud/master/%s'
DATA_URL = URL % 'lidar_chunks_1.csv'
LOOKUP_URL = URL % 'ground_truth_label.csv'
lidar = pd.read_csv(DATA_URL)
lookup = pd.read_csv(LOOKUP_URL)
lidar = lidar.merge(lookup)

In [ ]:
print('Number of points:', lidar.count())
lidar.head()

In [57]:
from pydeck import data_utils

color_lookup = data_utils.assign_random_colors(lidar['label_name'])

lidar['color'] = lidar.apply(lambda row: color_lookup.get(row['label_name']), axis=1)

lidar.head()

,x,y,z,label_id,confidence,label_name,color
0,-16.97,-27.84,5.72,1400,2,facade,"[197, 42, 104]"
1,-16.98,-27.83,5.42,1400,2,facade,"[197, 42, 104]"
2,-17.09,-28.00,5.73,1400,2,facade,"[197, 42, 104]"
3,-17.12,-28.00,5.42,1400,2,facade,"[197, 42, 104]"
4,-17.07,-27.98,5.14,1400,2,facade,"[197, 42, 104]"


In [58]:
from pydeck import Deck, Layer

lidar['coordinates'] = lidar.apply(lambda x: [x['x'], x['y'], x['z']], axis=1)

point_cloud = Layer(
    'PointCloudLayer',
    lidar[['coordinates', 'label_name', 'color']],
    get_position='coordinates',
    get_normal=[0, 0, 1],
    get_color='color',
    radius_pixels=4)

r = Deck(
    point_cloud,
    initial_view_state=pdk.ViewState(
        fov=2,
        rotation_x=0,
        max_zoom=100,
        coordinate_system='COORDINATE_SYSTEM.METERS',
        rotation_orbit=0,
        orbit_axis='Y',
        zoom=1,
        distance=10,
        min_distance=1,
        max_distance=100
    ),
    map_style=None
)
r.show()

DeckGLWidget(json_input='{"initialViewState": {"bearing": 0, "coordinateSystem": "COORDINATE_SYSTEM.METERS", "…